In [1]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer

In [2]:
qual_cols = { "age", "TSH", "T3", "TT4", "T4U", "FTI", "TBG" }
cat_cols = { "sex", "on_thyroxine", "on_antithyroid_meds", "sick", "pregnant", "thyroid_surgery", "I131_treatment", "lithium", "goitre", "tumor", "psych", "referral_source", "target" }

In [3]:
remove_qual_cols = { "TBG" }
remove_cat_cols = set()

In [4]:
k = 10
neighbours = 2
iters = 10000

In [5]:
seed = 42

In [6]:
random_state = np.random.RandomState(seed)
random_generator = np.random.default_rng(seed)

In [7]:
sel_qual_cols = list(qual_cols.difference(remove_qual_cols))
sel_cat_cols = list(cat_cols.difference(remove_cat_cols))
display(sel_qual_cols)
display(sel_cat_cols)

['TT4', 'T3', 'age', 'FTI', 'TSH', 'T4U']

['I131_treatment',
 'on_thyroxine',
 'sex',
 'on_antithyroid_meds',
 'referral_source',
 'thyroid_surgery',
 'psych',
 'target',
 'tumor',
 'sick',
 'goitre',
 'lithium',
 'pregnant']

In [8]:
df = pd.read_csv("./data/thyroidDF.csv")
df

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
0,29,F,f,f,f,f,f,f,f,t,...,NaN,f,NaN,f,NaN,f,NaN,other,-,840801013
1,29,F,f,f,f,f,f,f,f,f,...,128.0,f,NaN,f,NaN,f,NaN,other,-,840801014
2,41,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,11.0,other,-,840801042
3,36,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,26.0,other,-,840803046
4,32,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,36.0,other,S,840803047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9167,56,M,f,f,f,f,f,f,f,f,...,64.0,t,0.83,t,77.0,f,NaN,SVI,-,870119022
9168,22,M,f,f,f,f,f,f,f,f,...,91.0,t,0.92,t,99.0,f,NaN,SVI,-,870119023
9169,69,M,f,f,f,f,f,f,f,f,...,113.0,t,1.27,t,89.0,f,NaN,SVI,I,870119025
9170,47,F,f,f,f,f,f,f,f,f,...,75.0,t,0.85,t,88.0,f,NaN,other,-,870119027


In [9]:
df.dropna()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
167,40,F,f,f,f,f,f,f,f,f,...,3.9,t,0.83,t,5.0,t,28.0,other,F,840827019
5256,35,F,f,f,f,f,f,t,f,f,...,73.0,t,1.16,t,63.0,t,37.0,other,-,851128040
6044,77,F,f,f,f,f,f,f,f,f,...,120.0,t,0.96,t,124.0,t,45.0,SVI,-,860305064
6045,73,M,f,f,f,f,f,f,f,f,...,89.0,t,0.74,t,119.0,t,24.0,SVI,-,860305065
6747,77,F,f,f,f,f,f,f,f,f,...,131.0,t,1.04,t,126.0,t,25.0,SVI,K,860702030
6773,74,F,f,f,f,f,f,f,f,f,...,116.0,t,0.81,t,143.0,t,22.0,SVI,-,860703046
6862,60,M,f,f,f,f,f,f,f,f,...,92.0,t,0.84,t,110.0,t,21.0,other,-,860710043
6863,66,F,f,f,f,f,f,f,f,f,...,138.0,t,0.80,t,173.0,t,15.0,SVI,-,860710044
6880,42,F,f,f,f,f,f,f,f,f,...,106.0,t,0.98,t,108.0,t,27.0,other,-,860711039
6934,29,F,f,f,f,f,f,f,f,f,...,122.0,t,1.14,t,107.0,t,36.0,SVI,-,860717007


In [10]:
df[sel_qual_cols]

,TT4,T3,age,FTI,TSH,T4U
0,NaN,NaN,29,NaN,0.3,NaN
1,128.0,1.9,29,NaN,1.6,NaN
2,NaN,NaN,41,NaN,NaN,NaN
3,NaN,NaN,36,NaN,NaN,NaN
4,NaN,NaN,32,NaN,NaN,NaN
...,...,...,...,...,...,...
9167,64.0,NaN,56,77.0,NaN,0.83
9168,91.0,NaN,22,99.0,NaN,0.92
9169,113.0,NaN,69,89.0,NaN,1.27
9170,75.0,NaN,47,88.0,NaN,0.85


In [11]:
df[sel_cat_cols]

,I131_treatment,on_thyroxine,sex,on_antithyroid_meds,referral_source,thyroid_surgery,psych,target,tumor,sick,goitre,lithium,pregnant
0,f,f,F,f,other,f,f,-,f,f,f,f,f
1,f,f,F,f,other,f,f,-,f,f,f,f,f
2,f,f,F,f,other,f,f,-,f,f,f,f,f
3,f,f,F,f,other,f,f,-,f,f,f,f,f
4,f,f,F,f,other,f,f,S,f,f,f,f,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9167,f,f,M,f,SVI,f,f,-,f,f,f,f,f
9168,f,f,M,f,SVI,f,f,-,f,f,f,f,f
9169,f,f,M,f,SVI,f,f,I,f,f,f,f,f
9170,f,f,F,f,other,f,f,-,f,f,f,f,f


In [12]:
qual_na_df = df[sel_qual_cols].dropna()
qual_na_df

,TT4,T3,age,FTI,TSH,T4U
19,90.0,2.4,36,85.0,1.50,1.06
21,104.0,2.3,40,96.0,1.20,1.08
22,88.0,2.1,40,105.0,5.90,0.84
23,107.0,2.4,77,95.0,0.05,1.13
27,93.0,2.1,51,106.0,0.05,0.87
...,...,...,...,...,...,...
9129,85.0,1.8,65,94.0,0.73,0.90
9130,135.0,2.1,65,113.0,4.10,1.19
9134,49.0,1.0,74,39.0,53.00,1.25
9137,59.0,1.3,42,81.0,2.30,0.73


In [13]:
from data_split import k_fold_split

In [14]:
knn_imputed_df_map = dict()

complete_vals_df, remove_vals_df = k_fold_split(qual_na_df, k, random_generator)
missing_vals_idxs = list(remove_vals_df.index)

real_df = pd.concat([complete_vals_df, remove_vals_df])

for col in sel_qual_cols:
    missing_vals_df = remove_vals_df.copy()
    missing_vals_df[col] = np.NaN

    curr_df = pd.concat([complete_vals_df, missing_vals_df])

    knn_imputer = KNNImputer(n_neighbors=neighbours)
    knn_imputed_mat = knn_imputer.fit_transform(curr_df)
    
    knn_imputed_df = pd.DataFrame(knn_imputed_mat, columns=curr_df.columns, index=curr_df.index)
    knn_imputed_df["{} (real)".format(col)] = real_df[col]
    knn_imputed_df["{} (imputed)".format(col)] = knn_imputed_df[col]
    knn_imputed_df.drop([col], axis=1, inplace=True)

    knn_imputed_df = knn_imputed_df.loc[missing_vals_idxs]

    knn_imputed_df_map[col] = knn_imputed_df

In [15]:
for col, knn_imputed_df in knn_imputed_df_map.items():
    display("{} with KNN".format(col))
    display(knn_imputed_df)

'TT4 with KNN'

,T3,age,FTI,TSH,T4U,TT4 (real),TT4 (imputed)
92,0.4,88.0,134.0,0.20,0.73,98.0,114.5
312,0.2,65.0,17.0,145.00,0.94,16.0,24.5
1347,2.2,33.0,188.0,0.10,0.81,153.0,162.5
4525,1.9,85.0,145.0,0.58,0.96,140.0,135.0
4328,2.3,74.0,101.0,0.33,0.98,99.0,120.5
...,...,...,...,...,...,...,...
7528,2.5,55.0,85.0,2.30,1.09,93.0,78.5
2110,2.0,42.0,71.0,1.80,0.93,66.0,83.0
3210,1.9,61.0,97.0,1.80,1.06,102.0,93.5
4874,2.4,90.0,69.0,1.80,1.07,73.0,70.5


'T3 with KNN'

,TT4,age,FTI,TSH,T4U,T3 (real),T3 (imputed)
92,98.0,88.0,134.0,0.20,0.73,0.4,1.450
312,16.0,65.0,17.0,145.00,0.94,0.2,0.225
1347,153.0,33.0,188.0,0.10,0.81,2.2,1.650
4525,140.0,85.0,145.0,0.58,0.96,1.9,1.950
4328,99.0,74.0,101.0,0.33,0.98,2.3,1.300
...,...,...,...,...,...,...,...
7528,93.0,55.0,85.0,2.30,1.09,2.5,2.050
2110,66.0,42.0,71.0,1.80,0.93,2.0,2.400
3210,102.0,61.0,97.0,1.80,1.06,1.9,2.100
4874,73.0,90.0,69.0,1.80,1.07,2.4,1.400


'age with KNN'

...

,TT4,T3,age,TSH,T4U,FTI (real),FTI (imputed)
92,98.0,0.4,88.0,0.20,0.73,134.0,95.5
312,16.0,0.2,65.0,145.00,0.94,17.0,9.0
1347,153.0,2.2,33.0,0.10,0.81,188.0,123.5
4525,140.0,1.9,85.0,0.58,0.96,145.0,146.5
4328,99.0,2.3,74.0,0.33,0.98,101.0,90.5
...,...,...,...,...,...,...,...
7528,93.0,2.5,55.0,2.30,1.09,85.0,101.5
2110,66.0,2.0,42.0,1.80,0.93,71.0,73.5
3210,102.0,1.9,61.0,1.80,1.06,97.0,118.5
4874,73.0,2.4,90.0,1.80,1.07,69.0,66.5


'TSH with KNN'

,TT4,T3,age,FTI,T4U,TSH (real),TSH (imputed)
92,98.0,0.4,88.0,134.0,0.73,0.20,1.3000
312,16.0,0.2,65.0,17.0,0.94,145.00,55.0000
1347,153.0,2.2,33.0,188.0,0.81,0.10,0.2000
4525,140.0,1.9,85.0,145.0,0.96,0.58,1.3750
4328,99.0,2.3,74.0,101.0,0.98,0.33,0.6750
...,...,...,...,...,...,...,...
7528,93.0,2.5,55.0,85.0,1.09,2.30,3.1000
2110,66.0,2.0,42.0,71.0,0.93,1.80,0.6525
3210,102.0,1.9,61.0,97.0,1.06,1.80,0.8000
4874,73.0,2.4,90.0,69.0,1.07,1.80,1.6250


'T4U with KNN'

,TT4,T3,age,FTI,TSH,T4U (real),T4U (imputed)
92,98.0,0.4,88.0,134.0,0.20,0.73,0.745
312,16.0,0.2,65.0,17.0,145.00,0.94,1.020
1347,153.0,2.2,33.0,188.0,0.10,0.81,0.830
4525,140.0,1.9,85.0,145.0,0.58,0.96,0.965
4328,99.0,2.3,74.0,101.0,0.33,0.98,0.995
...,...,...,...,...,...,...,...
7528,93.0,2.5,55.0,85.0,2.30,1.09,1.070
2110,66.0,2.0,42.0,71.0,1.80,0.93,0.940
3210,102.0,1.9,61.0,97.0,1.80,1.06,1.050
4874,73.0,2.4,90.0,69.0,1.80,1.07,1.060


In [16]:
mice_imputed_df_map = dict()

complete_vals_df, remove_vals_df = k_fold_split(qual_na_df, k, random_generator)
missing_vals_idxs = list(remove_vals_df.index)

real_df = pd.concat([complete_vals_df, remove_vals_df])

for col in sel_qual_cols:
    missing_vals_df = remove_vals_df.copy()
    missing_vals_df[col] = np.NaN

    curr_df = pd.concat([complete_vals_df, missing_vals_df])

    mice_imputer = IterativeImputer(random_state=random_state, max_iter=iters)
    mice_imputed_mat = mice_imputer.fit_transform(curr_df)
    
    mice_imputed_df = pd.DataFrame(mice_imputed_mat, columns=curr_df.columns, index=curr_df.index)
    mice_imputed_df["{} (real)".format(col)] = real_df[col]
    mice_imputed_df["{} (imputed)".format(col)] = mice_imputed_df[col]
    mice_imputed_df.drop([col], axis=1, inplace=True)

    mice_imputed_df = mice_imputed_df.loc[missing_vals_idxs]

    mice_imputed_df_map[col] = mice_imputed_df

In [17]:
for col, mice_imputed_df in mice_imputed_df_map.items():
    display("{} with MICE".format(col))
    display(mice_imputed_df)

'TT4 with MICE'

,T3,age,FTI,TSH,T4U,TT4 (real),TT4 (imputed)
734,2.0,27.0,95.0,5.60,1.41,134.0,140.687933
6971,1.1,81.0,131.0,13.00,0.84,110.0,104.474669
4563,2.7,70.0,88.0,2.30,1.07,95.0,101.141237
3350,2.0,73.0,97.0,0.84,1.04,101.0,103.624700
2235,1.6,74.0,89.0,0.60,1.07,95.0,100.133954
...,...,...,...,...,...,...,...
916,0.7,74.0,89.0,0.60,0.77,68.0,66.689368
6829,2.8,46.0,144.0,1.20,1.26,181.0,163.486578
603,1.9,69.0,115.0,0.40,0.92,106.0,104.236303
8241,2.3,46.0,105.0,1.20,0.93,98.0,98.433522


'T3 with MICE'

,TT4,age,FTI,TSH,T4U,T3 (real),T3 (imputed)
734,134.0,27.0,95.0,5.60,1.41,2.0,2.653621
6971,110.0,81.0,131.0,13.00,0.84,1.1,1.847232
4563,95.0,70.0,88.0,2.30,1.07,2.7,1.892578
3350,101.0,73.0,97.0,0.84,1.04,2.0,1.932987
2235,95.0,74.0,89.0,0.60,1.07,1.6,1.901187
...,...,...,...,...,...,...,...
916,68.0,74.0,89.0,0.60,0.77,0.7,1.299452
6829,181.0,46.0,144.0,1.20,1.26,2.8,2.966864
603,106.0,69.0,115.0,0.40,0.92,1.9,1.880045
8241,98.0,46.0,105.0,1.20,0.93,2.3,1.794005


'age with MICE'

...

,TT4,T3,age,TSH,T4U,FTI (real),FTI (imputed)
734,134.0,2.0,27.0,5.60,1.41,95.0,83.938754
6971,110.0,1.1,81.0,13.00,0.84,131.0,131.273540
4563,95.0,2.7,70.0,2.30,1.07,88.0,88.409761
3350,101.0,2.0,73.0,0.84,1.04,97.0,96.701175
2235,95.0,1.6,74.0,0.60,1.07,89.0,85.229518
...,...,...,...,...,...,...,...
916,68.0,0.7,74.0,0.60,0.77,89.0,93.227539
6829,181.0,2.8,46.0,1.20,1.26,144.0,155.645817
603,106.0,1.9,69.0,0.40,0.92,115.0,117.470016
8241,98.0,2.3,46.0,1.20,0.93,105.0,108.776557


'TSH with MICE'

,TT4,T3,age,FTI,T4U,TSH (real),TSH (imputed)
734,134.0,2.0,27.0,95.0,1.41,5.60,13.278634
6971,110.0,1.1,81.0,131.0,0.84,13.00,2.535249
4563,95.0,2.7,70.0,88.0,1.07,2.30,9.379132
3350,101.0,2.0,73.0,97.0,1.04,0.84,8.362239
2235,95.0,1.6,74.0,89.0,1.07,0.60,11.683056
...,...,...,...,...,...,...,...
916,68.0,0.7,74.0,89.0,0.77,0.60,9.184800
6829,181.0,2.8,46.0,144.0,1.26,1.20,-6.587121
603,106.0,1.9,69.0,115.0,0.92,0.40,3.705222
8241,98.0,2.3,46.0,105.0,0.93,1.20,4.894284


'T4U with MICE'

,TT4,T3,age,FTI,TSH,T4U (real),T4U (imputed)
734,134.0,2.0,27.0,95.0,5.60,1.41,1.228634
6971,110.0,1.1,81.0,131.0,13.00,0.84,0.875253
4563,95.0,2.7,70.0,88.0,2.30,1.07,1.045581
3350,101.0,2.0,73.0,97.0,0.84,1.04,1.012983
2235,95.0,1.6,74.0,89.0,0.60,1.07,1.006599
...,...,...,...,...,...,...,...
916,68.0,0.7,74.0,89.0,0.60,0.77,0.816409
6829,181.0,2.8,46.0,144.0,1.20,1.26,1.276063
603,106.0,1.9,69.0,115.0,0.40,0.92,0.945619
8241,98.0,2.3,46.0,105.0,1.20,0.93,0.961926


In [18]:
from proba_utils import cum_sum_intervals_from_weights, sample_cols, create_nan_vals

In [19]:
weight_map = {
    "age" : 1,
    "T3"  : 1,
    "T4U" : 1,
    "TSH" : 1,
    "TT4" : 1,
    "FTI" : 1
}

cum_sum_intervals, cum_sum_col_map = cum_sum_intervals_from_weights(weight_map)

display(cum_sum_intervals)
display(cum_sum_col_map)

[Interval(0, 0.16666666666666666, closed='left'),
 Interval(0.16666666666666666, 0.3333333333333333, closed='left'),
 Interval(0.3333333333333333, 0.5, closed='left'),
 Interval(0.5, 0.6666666666666666, closed='left'),
 Interval(0.6666666666666666, 0.8333333333333333, closed='left'),
 Interval(0.8333333333333333, 1, closed='both')]

{0: 'age', 1: 'T3', 2: 'T4U', 3: 'TSH', 4: 'TT4', 5: 'FTI'}

In [20]:
complete_vals_df, remove_vals_df = k_fold_split(qual_na_df, k, random_generator)

picked_cols = sample_cols(remove_vals_df.shape[0], cum_sum_intervals, cum_sum_col_map, random_generator)
missing_vals_df, missing_col_map = create_nan_vals(remove_vals_df, picked_cols)

missing_vals_idxs = list(missing_vals_df.index)

real_df = pd.concat([complete_vals_df, remove_vals_df])
curr_df = pd.concat([complete_vals_df, missing_vals_df])

knn_imputer = KNNImputer(n_neighbors=neighbours)
knn_imputed_mat = knn_imputer.fit_transform(curr_df)

knn_imputed_df = pd.DataFrame(knn_imputed_mat, columns=real_df.columns, index=real_df.index)
knn_imputed_df = knn_imputed_df.loc[missing_vals_idxs]

for col in remove_vals_df.columns:
    knn_imputed_df["{} (real)".format(col)] = remove_vals_df[col]
    knn_imputed_df["{} (imputed)".format(col)] = knn_imputed_df[col]
    knn_imputed_df.drop([col], axis=1, inplace=True)

knn_imputed_df["imputed"] = picked_cols

knn_imputed_df

,TT4 (real),TT4 (imputed),T3 (real),T3 (imputed),age (real),age (imputed),FTI (real),FTI (imputed),TSH (real),TSH (imputed),T4U (real),T4U (imputed),imputed
6880,106.0,95.5,2.5,2.50,42.0,42.0,108.0,108.0,1.200,1.200,0.98,0.980,TT4
2987,153.0,153.0,2.0,2.00,66.0,58.0,154.0,154.0,0.065,0.065,0.99,0.990,age
7669,5.0,5.0,0.3,0.30,61.0,61.0,4.5,4.5,106.000,106.000,1.11,0.945,T4U
1364,72.0,72.0,1.5,1.50,53.0,53.0,69.0,77.5,1.500,1.500,1.06,1.060,FTI
965,114.0,114.0,1.1,3.75,58.0,58.0,124.0,124.0,1.400,1.400,0.92,0.920,T3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8028,175.0,118.0,5.1,5.10,45.0,45.0,192.0,192.0,3.500,3.500,0.91,0.910,TT4
4433,76.0,76.0,1.4,1.40,82.0,82.0,88.0,88.0,3.200,4.000,0.86,0.860,TSH
3469,170.0,170.0,2.5,2.50,37.0,45.5,121.0,121.0,0.540,0.540,1.40,1.400,age
2864,103.0,100.0,1.7,1.70,44.0,44.0,107.0,107.0,3.500,3.500,0.96,0.960,TT4


In [21]:
complete_vals_df, remove_vals_df = k_fold_split(qual_na_df, k, random_generator)

picked_cols = sample_cols(remove_vals_df.shape[0], cum_sum_intervals, cum_sum_col_map, random_generator)
missing_vals_df, missing_col_map = create_nan_vals(remove_vals_df, picked_cols)

missing_vals_idxs = list(missing_vals_df.index)

real_df = pd.concat([complete_vals_df, remove_vals_df])
curr_df = pd.concat([complete_vals_df, missing_vals_df])

mice_imputer = IterativeImputer(random_state=random_state, max_iter=iters)
mice_imputed_mat = mice_imputer.fit_transform(curr_df)

mice_imputed_df = pd.DataFrame(mice_imputed_mat, columns=real_df.columns, index=real_df.index)
mice_imputed_df = mice_imputed_df.loc[missing_vals_idxs]

for col in remove_vals_df.columns:
    mice_imputed_df["{} (real)".format(col)] = remove_vals_df[col]
    mice_imputed_df["{} (imputed)".format(col)] = mice_imputed_df[col]
    mice_imputed_df.drop([col], axis=1, inplace=True)

mice_imputed_df["imputed"] = picked_cols

mice_imputed_df

,TT4 (real),TT4 (imputed),T3 (real),T3 (imputed),age (real),age (imputed),FTI (real),FTI (imputed),TSH (real),TSH (imputed),T4U (real),T4U (imputed),imputed
3269,101.0,101.000000,1.8,1.800000,76.0,76.000000,86.0,86.000000,1.30,1.300000,1.17,1.068989,T4U
31,47.0,47.000000,0.1,0.100000,85.0,85.000000,69.0,81.080282,0.05,0.050000,0.68,0.680000,FTI
3828,92.0,92.000000,1.4,1.400000,72.0,72.000000,93.0,93.000000,5.70,9.943933,0.99,0.990000,TSH
990,87.0,87.000000,1.2,1.581716,73.0,73.000000,104.0,104.000000,2.70,2.700000,0.83,0.830000,T3
5729,77.0,77.000000,0.5,0.500000,39.0,39.000000,70.0,70.000000,1.70,19.493308,1.10,1.100000,TSH
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4269,101.0,101.000000,2.3,2.300000,65.0,75.840077,106.0,106.000000,1.70,1.700000,0.95,0.950000,age
523,84.0,84.000000,2.0,2.000000,72.0,72.000000,111.0,111.000000,0.60,0.600000,0.76,0.831753,T4U
4122,17.0,17.000000,0.4,0.400000,50.0,50.000000,15.0,15.000000,117.00,37.031455,1.14,1.140000,TSH
7446,105.0,112.680545,2.7,2.700000,65.0,65.000000,91.0,91.000000,2.80,2.800000,1.16,1.160000,TT4
